In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

import config

In [2]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import pandas as pd
import json
from tqdm import tqdm

In [3]:
with open("../../data/player_summaries.json", "r", encoding="utf-8") as f:
    players_summaries = json.load(f)
    
df_summaries = pd.DataFrame([
    {'player': player.split('(')[0].strip(), 'team': player.split('(')[1].replace(')', '').strip(), 'summary': summary} for player, summary in players_summaries.items()
])
df_summaries = df_summaries.reset_index() 
df_summaries.head()

,index,player,team,summary
0,0,Ben White,Arsenal,Le joueur évolue principalement en tant que dé...
1,1,Bukayo Saka,Arsenal,Le joueur évolue principalement en tant qu'att...
2,2,David Raya,Arsenal,Le joueur évolue principalement au poste de ga...
3,3,Declan Rice,Arsenal,Le joueur évolue principalement au poste de mi...
4,4,Ethan Nwaneri,Arsenal,Le joueur évolue principalement en tant qu'att...


In [4]:
df_players = pd.read_csv('../../data/players_stats.csv')
df_players = df_players.reset_index(drop=True)
df_players.head()

,Unnamed: 0,index,league,season,team,player,nation__standard,pos__standard,age__standard,born__standard,...,Performance_Crs_misc,Performance_Int_misc,Performance_TklW_misc,Performance_PKwon_misc,Performance_PKcon_misc,Performance_OG_misc,Performance_Recov_misc,Aerial Duels_Won_misc,Aerial Duels_Lost_misc,Aerial Duels_Won%_misc
0,0,0,ENG-Premier League,2425,Arsenal,Ben White,ENG,DF,26.0,1997.0,...,17,16,14,0,0,0,37,18,9,66.7
1,1,1,ENG-Premier League,2425,Arsenal,Bukayo Saka,ENG,"FW,MF",22.0,2001.0,...,117,3,15,1,0,0,70,10,20,33.3
2,2,2,ENG-Premier League,2425,Arsenal,David Raya,ESP,GK,28.0,1995.0,...,0,2,0,0,1,0,41,13,0,100.0
3,3,3,ENG-Premier League,2425,Arsenal,Declan Rice,ENG,MF,25.0,1999.0,...,164,25,25,0,0,0,156,37,25,59.7
4,4,4,ENG-Premier League,2425,Arsenal,Ethan Nwaneri,ENG,"FW,MF",17.0,2007.0,...,54,2,7,0,0,0,34,4,8,33.3


In [5]:
df_players_summaries = df_summaries.merge(df_players, how='left', on=['player', 'team'])
df_players_summaries = df_players_summaries = df_players_summaries[[
    'league',
    'season',
    'player',
    'team',
    'pos__standard',
    'summary'
]]

df_players_summaries.rename(columns={'pos__standard': 'position'}, inplace=True)
df_players_summaries.head()

,league,season,player,team,position,summary
0,ENG-Premier League,2425,Ben White,Arsenal,DF,Le joueur évolue principalement en tant que dé...
1,ENG-Premier League,2425,Bukayo Saka,Arsenal,"FW,MF",Le joueur évolue principalement en tant qu'att...
2,ENG-Premier League,2425,David Raya,Arsenal,GK,Le joueur évolue principalement au poste de ga...
3,ENG-Premier League,2425,Declan Rice,Arsenal,MF,Le joueur évolue principalement au poste de mi...
4,ENG-Premier League,2425,Ethan Nwaneri,Arsenal,"FW,MF",Le joueur évolue principalement en tant qu'att...


In [6]:
embedding_model = SentenceTransformer("BAAI/bge-m3")
qdrant_client = QdrantClient(
    host='localhost', 
    port=6333,
    timeout=30.0
)

In [7]:
collection_name = 'ragscout_players'

if not qdrant_client.collection_exists(collection_name):
    qdrant_client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(
            size=1024,
            distance=Distance.COSINE
        )
    )

In [8]:
points = []
for idx, row in tqdm(df_players_summaries.iterrows()):
    embedding = embedding_model.encode(row['summary']).tolist()
    
    metadata = {
        'season': row['season'],
        'player': row['player'],
        'league': row['league'],
        'team': row['team'],
        'position': row['position'],
        'summary': row['summary']
    }
    
    point = PointStruct(
        id=idx,
        vector=embedding,
        payload=metadata
    )
    
    points.append(point)

0it [00:00, ?it/s]/Users/florian/.pyenv/versions/scout_rag_env/lib/python3.10/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
1735it [03:53,  7.44it/s]


In [9]:
BATCH_SIZE = 100

for i in range(0, len(points), BATCH_SIZE):
    batch = points[i:i + BATCH_SIZE]
    qdrant_client.upsert(
        collection_name=collection_name, 
        points=batch
    )
    
    print(f"✅ {min(i + BATCH_SIZE, len(points))} / {len(points)} envoyé")

print(f"✅ {len(points)} joueurs insérés dans Qdrant avec succès !")

✅ 100 / 1735 envoyé
✅ 200 / 1735 envoyé
✅ 300 / 1735 envoyé
✅ 400 / 1735 envoyé
✅ 500 / 1735 envoyé
✅ 600 / 1735 envoyé
✅ 700 / 1735 envoyé
✅ 800 / 1735 envoyé
✅ 900 / 1735 envoyé
✅ 1000 / 1735 envoyé
✅ 1100 / 1735 envoyé
✅ 1200 / 1735 envoyé
✅ 1300 / 1735 envoyé
✅ 1400 / 1735 envoyé
✅ 1500 / 1735 envoyé
✅ 1600 / 1735 envoyé
✅ 1700 / 1735 envoyé
✅ 1735 / 1735 envoyé
✅ 1735 joueurs insérés dans Qdrant avec succès !


In [10]:
query = 'Ailier dynamique, fort impact des buts et passe décisives de son equipe, fort à la finition. Rapide, dribbleur, avec pas mal de passes clés'
query_vector = embedding_model.encode(query).tolist()

results = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=5
)

for hit in results:
    print(f"🔹 {hit.payload['player']} ({hit.payload['team']})")
    print(f"Résumé : {hit.payload['summary']}")
    print(f"Score : {hit.score:.3f}")
    print("---")

🔹 Brooke Norton-Cuffy (Genoa)
Résumé : Le joueur évolue principalement en tant que défenseur, avec la possibilité d'occuper un rôle d'ailier offensif. Ses statistiques indiquent une implication significative dans les phases défensives, avec un bon volume de jeu et une capacité à réaliser des interventions efficaces. Il se montre actif dans la récupération du ballon, avec un bon taux de réussite dans ses tacles et une présence notable dans les duels aériens. En phase offensive, il participe à la construction du jeu, bien qu'il n'ait pas encore réussi à marquer ou à délivrer des passes décisives.

Ses principales qualités se répartissent sur plusieurs axes. Sur le plan physique, il affiche un bon volume de jeu avec une présence dans les duels, bien qu'il montre quelques limites dans les duels aériens. Techniquement, il est capable de réaliser des passes précises, notamment à courte et moyenne distance, mais il pourrait améliorer son efficacité sur les passes longues. Mentalement, il démo

/var/folders/x9/t16jfwq91z735wm10ntxvh780000gq/T/ipykernel_69994/4214100557.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(
